In [ ]:
import numpy as np
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit import IBMQ, Aer, transpile, assemble, execute
from qiskit.visualization import plot_histogram, plot_bloch_multivector, array_to_latex
from qiskit.extensions import Initialize
from qiskit.quantum_info import random_statevector, Statevector
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit.test.reference_circuits import ReferenceCircuits
from qiskit_ibm_runtime import QiskitRuntimeService, Sampler
import pylatexenc
import matplotlib.pyplot as plt
%matplotlib inline



In [ ]:
def entangle(qc, a, b):
    qc.h(a)
    qc.cx(a,b)
    
def measure(qc, a, b):
    qc.measure(a, 0)
    qc.measure(b, 1)
    
def alice_gate(qc, psi, a):
    qc.cx(psi, a)
    qc.h(psi)
    
def bob_gate(qc, qubit, crz, crx):
    qc.x(qubit).c_if(crx, 1)
    qc.z(qubit).c_if(crz, 1)

In [ ]:
psi = Statevector([0.2861820382  -0.3387958088j, -0.7129244311 -0.5431905712j])
#psi = random_statevector(2)
display(array_to_latex(psi, prefix="|\\psi\\rangle ="))
plot_bloch_multivector(psi)

init_gate = Initialize(psi)

qr = QuantumRegister(3, name="q" )
classic_z = ClassicalRegister(1, name="crx")
classic_x = ClassicalRegister(1, name="crz")
classic_b = ClassicalRegister(1, name="crb")
qc = QuantumCircuit(qr, classic_x, classic_z, classic_b)


qc.append(init_gate, [0])
entangle(qc, 1, 2)
qc.barrier(0,1,2)
alice_gate(qc, 0, 1)
qc.barrier(0,1,2)
measure(qc, 0, 1)
qc.barrier(0,1,2)
bob_gate(qc, 2, classic_x, classic_z)
qc.measure(2, 2)
qc.draw(output="mpl")

In [ ]:
sim = Aer.get_backend('aer_simulator')
result = execute(qc.reverse_bits(), sim, shots=1000)
counts = result.result().get_counts()

fig, ax = plt.subplots()
plot_histogram(counts, ax=ax)
ax.set_xlabel('Measurement outcome')
ax.set_ylabel('Frequency')
ax.yaxis.grid(color='black', linestyle='-', linewidth=1)
ax.set_yticks(range(0, 300, 50))
plt.xticks(rotation='horizontal')
ax.spines['right'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.yaxis.grid(True, linestyle='-', linewidth=1)

y_min = 0
y_max = 310
bar_width = 0.5

for i, (_, count) in enumerate(counts.items()):
    ax.fill_between([i - bar_width/2, i + bar_width/2], y_min, y_max, color='lightgray')


# Mostre o histograma
plt.show()


In [ ]:
bob_qubit=0
from qiskit.result import marginal_counts
bob_counts = marginal_counts(result.result().get_counts(), [bob_qubit])
plot_histogram(bob_counts)
fig, ax = plt.subplots()
plot_histogram(bob_counts, ax=ax)
ax.set_xlabel('Measurement outcome')
ax.set_ylabel('Frequency')
ax.yaxis.grid(color='black', linestyle='-', linewidth=1)
ax.set_yticks(range(0, 1000, 100))
plt.xticks(rotation='horizontal')
ax.spines['right'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.yaxis.grid(True, linestyle='-', linewidth=1)

y_min = 0
y_max = 900
bar_width = 0.5

for i, (_, count) in enumerate(bob_counts.items()):
    ax.fill_between([i - bar_width/2, i + bar_width/2], y_min, y_max, color='lightgray')


# Mostre o histograma
plt.show()

